In [7]:
!pip install snowflake-connector-python snowflake-sqlalchemy pyspark

!pip install snowflake-snowpark-python




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 3.0.0
    Uninstalling cloudpickle-3.0.0:
      Successfully uninstalled cloudpickle-3.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2024.11.2 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import snowflake.connector
print(snowflake.connector.__version__)


3.13.2


In [11]:
from snowflake.snowpark import Session
import pandas as pd

# Snowflake connection parameters
connection_parameters = {
    "account": "VQNCMPN-OO50404",
    "user": "SAIV22",
    "password": "PalakurthiSai22",
    "warehouse": "COMPUTE_WH",
    "database": "FRAUDDETECTIONDB",
    "schema": "PUBLIC"
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

# Read data from Snowflake table
fraud_df = session.table("transactions_snowflake_New_Three").to_pandas()


In [15]:
print(fraud_df.describe)

<bound method NDFrame.describe of                   CC_NUM                              MERCHANT  \
0       2291160000000000                 fraud_Kirlin and Sons   
1       3573030000000000                  fraud_Sporer-Keebler   
2       3598220000000000  fraud_Swaniawski, Nitzsche and Welch   
3       3591920000000000                     fraud_Haley Group   
4       3526830000000000                 fraud_Johnston-Casper   
...                  ...                                   ...   
555714    30560600000000                 fraud_Reilly and Sons   
555715  3556610000000000                  fraud_Hoppe-Parisian   
555716  6011720000000000                       fraud_Rau-Robel   
555717     4079770000000                 fraud_Breitenberg LLC   
555718  4170690000000000                     fraud_Dare-Marvin   

              CATEGORY     AMT    FIRST      LAST GENDER  \
0        personal_care    2.86     Jeff   Elliott      M   
1        personal_care   29.84   Joanne  Williams    

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Select features and target variable
X = fraud_df[['AMT', 'CITY_POP', 'MERCH_LAT', 'MERCH_LONG']]
y = fraud_df['IS_FRAUD']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, f1_score

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate model performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("f1_score:", f1_score(y_test,y_pred))
print("Precision score:" , precision_score(y_test,y_pred))



Accuracy: 0.9967339667458432
f1_score: 0.40589198036006546
Precision score: 0.6702702702702703
